In [ ]:
# Установка необходимых библиотек
!pip install conllu zss transformers numpy scipy

import conllu
from zss import simple_distance, Node
import numpy as np
from scipy.spatial.distance import cosine
from transformers import BertTokenizer, BertModel
import torch

# ======================
# 5.1 Объяснение UD-тегов на примере из корпуса
# ======================
!wget https://www.dropbox.com/s/am6nasx6bx82nhp/RuEval2017-Lenta-news-dev.conllu
!head -497 RuEval2017-Lenta-news-dev.conllu | tail -30 > sample.conllu

with open('sample.conllu', 'r', encoding='utf-8') as f:
    sample_data = f.read()

parsed_sample = conllu.parse(sample_data)

def explain_ud_tags(parsed_data):
    print("Примеры UD-тегов из корпуса:")

    # Выбираем интересные теги для демонстрации
    tags_to_explain = ['nsubj', 'obj', 'conj']
    examples_found = {tag: None for tag in tags_to_explain}

    for sentence in parsed_data:
        for token in sentence:
            if token['deprel'] in tags_to_explain and not examples_found[token['deprel']]:
                head_word = next(t['form'] for t in sentence if t['id'] == token['head'])
                examples_found[token['deprel']] = (token['form'], head_word, token['deprel'])

        if all(examples_found.values()):
            break

    for tag, example in examples_found.items():
        if example:
            print(f"\n{tag.upper()}:")
            print(f"Пример: '{example[0]}' зависит от '{example[1]}' с отношением {example[2]}")
            if tag == 'nsubj':
                print("Это подлежащее в предложении (номинативный субъект)")
            elif tag == 'obj':
                print("Это прямой объект действия (прямое дополнение)")
            elif tag == 'conj':
                print("Это сочинительная связь между однородными членами")

explain_ud_tags(parsed_sample)

  Using cached conllu-6.0.0-py3-none-any.whl.metadata (21 kB)
  Using cached zss-1.2.0.tar.gz (9.8 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for zss: filename=zss-1.2.0-py3-none-any.whl size=6725 sha256=549220ba95fb6e5a0a4f69cc9e3b40ad634dae53096527f05081f30d2b7bb181
  Stored in directory: /root/.cache/pip/wheels/e8/3a/21/9f03c8002063200de26717429ca1543466c5feec134a0a73ff
Successfully built zss
--2025-05-07 14:06:05--  https://www.dropbox.com/s/am6nasx6bx82nhp/RuEval2017-Lenta-news-dev.conllu
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/xd41e0habfswo5uchljpt/RuEval2017-Lenta-news-dev.conllu?rlkey=0x30o9bdarkbr0re5xcxn9sg4 [following]
--2025-05-07 14:06:06--  https://www.dropbox.com/scl/fi/xd41e0habfswo5uchljpt/RuEval2017-Lenta-news-dev.conllu?rlkey=0x30o9bd

In [ ]:
# 2. Функция разбиения сложносочиненного предложения на простые
# ============================================================
def split_compound_sentence(sentence, conllu_data):
    token_list = conllu_data[0]  # берем первую (и единственную) запись
    conj_indices = [i for i, token in enumerate(token_list)
                   if token['deprel'] == 'conj']

    if not conj_indices:
        return [sentence]

    # Найдем корень предложения
    root = [token for token in token_list if token['deprel'] == 'root'][0]
    root_id = root['id']

    # Разделим предложение на части
    parts = []
    current_part = []

    for token in token_list:
        current_part.append(token['form'])
        if token['id'] in conj_indices or token['head'] == root_id and token['deprel'] == 'cc':
            parts.append(' '.join(current_part))
            current_part = []

    if current_part:
        parts.append(' '.join(current_part))

    return parts

# Пример использования
sample_conllu = """
1	Кот	кот	NOUN	_	Case=Nom|Gender=Masc	2	nsubj	_	_
2	спит	спать	VERB	_	Aspect=Imp|Number=Sing	0	root	_	_
3	,	,	PUNCT	_	_	4	punct	_	_
4	а	а	CCONJ	_	_	2	cc	_	_
5	собака	собака	NOUN	_	Case=Nom|Gender=Fem	6	nsubj	_	_
6	лает	лаять	VERB	_	Aspect=Imp|Number=Sing	2	conj	_	_
"""

parsed_data = conllu.parse(sample_conllu)
sentence = "Кот спит, а собака лает"
print("\nРазбиение предложения:", split_compound_sentence(sentence, parsed_data))


Разбиение предложения: ['Кот спит , а', 'собака', 'лает']


In [ ]:
# 3. Наименьший общий предок в дереве зависимостей
# =============================================
def find_lca(token1_id, token2_id, conllu_data):
    token_list = conllu_data[0]
    ancestors1 = set()
    ancestors2 = set()

    # Собираем предков первого токена
    current_id = token1_id
    while current_id != 0:
        ancestors1.add(current_id)
        current_id = next(token['head'] for token in token_list if token['id'] == current_id)

    # Собираем предков второго токена
    current_id = token2_id
    while current_id != 0:
        ancestors2.add(current_id)
        current_id = next(token['head'] for token in token_list if token['id'] == current_id)

    # Находим пересечение
    common_ancestors = ancestors1 & ancestors2
    if not common_ancestors:
        return None

    return min(common_ancestors)  # самый нижний из общих предков

# Пример использования
token1_id = 1  # "Кот"
token2_id = 5  # "собака"
print("\nНаименьший общий предок:", find_lca(token1_id, token2_id, parsed_data))


Наименьший общий предок: 2


In [ ]:
# 5.4 Сравнение предложений двумя методами
# =============================================
examples = [
    "Привет, у нас на кухне нашли плесень!",
    "На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.",
    "Привет, у них в подвале нашли клад!"
]

# Метод 1: Расстояние редактирования деревьев зависимостей (zss)
# Для простоты будем использовать упрощенное представление деревьев

def build_simple_tree(sentence):
    # В реальной задаче нужно использовать синтаксический анализатор
    # Здесь просто эмуляция для демонстрации
    words = sentence.split()
    root = Node('root')
    nodes = {}

    # Создаем узлы для каждого слова
    for i, word in enumerate(words):
        nodes[i] = Node(word)

    # Простая эмуляция дерева зависимостей
    if len(words) > 0:
        root.addkid(nodes[0])
        for i in range(1, len(words)):
            parent = max(0, i-2)  # Простая эвристика для демонстрации
            nodes[parent].addkid(nodes[i])

    return root

def tree_similarity(sent1, sent2):
    tree1 = build_simple_tree(sent1)
    tree2 = build_simple_tree(sent2)
    distance = simple_distance(tree1, tree2)
    return 1 / (1 + distance)  # Преобразуем расстояние в меру сходства

print("\nСравнение предложений методом zss (расстояние редактирования деревьев):")
for i in range(len(examples)):
    for j in range(i+1, len(examples)):
        sim = tree_similarity(examples[i], examples[j])
        print(f"Сходство между '{examples[i]}' и '{examples[j]}': {sim:.3f}")

# Метод 2: Косинусная мера между BERT-эмбеддингами
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence')
model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased-sentence')

def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return torch.mean(outputs.last_hidden_state, dim=1).squeeze().numpy()

# Вычисляем эмбеддинги для всех примеров
embeddings = [get_bert_embedding(sent) for sent in examples]

print("\nСравнение предложений методом BERT-эмбеддингов:")
for i in range(len(examples)):
    for j in range(i+1, len(examples)):
        cos_sim = 1 - cosine(embeddings[i], embeddings[j])
        print(f"Косинусное сходство между '{examples[i]}' и '{examples[j]}': {cos_sim:.3f}")


Сравнение предложений методом zss (расстояние редактирования деревьев):
Сходство между 'Привет, у нас на кухне нашли плесень!' и 'На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.': 0.016
Сходство между 'Привет, у нас на кухне нашли плесень!' и 'Привет, у них в подвале нашли клад!': 0.059
Сходство между 'На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.' и 'Привет, у них в подвале нашли клад!': 0.016


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/711M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]


Сравнение предложений методом BERT-эмбеддингов:
Косинусное сходство между 'Привет, у нас на кухне нашли плесень!' и 'На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.': 0.788
Косинусное сходство между 'Привет, у нас на кухне нашли плесень!' и 'Привет, у них в подвале нашли клад!': 0.640
Косинусное сходство между 'На нашей кухне нашли много всего: бактерии, грибки и позавчерашнее молоко.' и 'Привет, у них в подвале нашли клад!': 0.543
